<a href="https://colab.research.google.com/github/AliAkbarBadri/graph-centrality/blob/main/axiom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prerequisite

In [8]:
import networkx as nx
import pandas as pd
import glob
import time
import numpy as np
from random import choice
from random import sample

In [4]:

def count_subgraph(G, v):
  neighbours = G.edges(v)
  if len(neighbours) == 0:
    return 1
  else:
    for e in neighbours:
      G_temp = G.copy()
      G_temp.remove_edge(*e)
      a = count_subgraph(G_temp, v)
      b = (2**G.number_of_edges(*e) - 1)*count_subgraph(nx.contracted_edge(G, e, self_loops=False), e[0])
      return a + b
def count_tree(T, v):
  neighbours = T.edges(v)
  if len(neighbours) == 0:
    return 1
  else:
    for e in neighbours:
      T_temp = T.copy()
      T_temp.remove_edge(*e)
      return count_tree(T_temp, v) * (count_tree(T_temp,e[1]) + 1)

! curl -L https://api.github.com/repos/AliAkbarBadri/graph-centrality/tarball --output repo.tar
! tar xf repo.tar --wildcards "*/data/*.csv" --strip-components=1
! rm -rf repo.tar

def create_graph_from_csv(dir_name:str) -> dict:
  graphs = {}
  for file_name in sorted(glob.glob(dir_name+'/*.csv')):
    df = pd.read_csv(file_name,header=None,names=['src', 'dest'])
    G = nx.from_pandas_edgelist(df,source="src",target="dest", create_using=nx.DiGraph())
    dataset_name = file_name.split("/")[1].split(".csv")[0]
    print(dataset_name, G.number_of_nodes(), G.number_of_edges())
    graphs[dataset_name] = G
  return graphs

graphs = create_graph_from_csv(dir_name="data")

  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  438k    0  438k    0     0   632k      0 --:--:-- --:--:-- --:--:-- 1365k
ENZYMES_g297 121 298
aves-songbird-social 110 1027
bn-mouse_visual-cortex_2 193 214
ca-sandi_auths 86 124
email-dnc-corecipient 906 12085
email-enron-only 143 623
facebook-socfb-Amherst41 2235 90953
insecta-ant-colony1-day01 113 4550
reptilia-tortoise-network-lm 45 106
road-chesapeake 39 170
rt-retweet 96 117
soc-dolphins 62 159
soc-wiki-Vote 889 2914
web-edu 3031 6474
web-polblogs 643 2280
web-spam 4767 37375


axiom 1 :monotonicity

In [5]:
def check_monotonicity(G, number_node):
    # axiom 1 : monotonicity
    # test C(v,G-e) <= C(v,G) for 10 random node
    check_array = [0, 0]
    for i in range(number_node):
        random_node = choice(list(G.nodes()))
        removed_edge = choice(list(G.edges()))
        score_v = count_tree(G, random_node)
        G_temp = G.copy()
        G_temp.remove_edge(*removed_edge)
        score_v_without = count_tree(G_temp, random_node)
        check = score_v_without <= score_v
        if check:
            check_array[0] += 1
        else:
            check_array[1] += 1
        print(str(i) + "st round : compute score for node " + str(random_node) + " , remove " + str(
            removed_edge) + " edge , meet condition? " + str(check))
    print("Success rate =  " + str(round((check_array[0] / (check_array[0] + check_array[1])) * 100)) + "%")


test axiom 1

In [7]:
number_node = 10
for key, value in graphs.items():
  if graphs[key].number_of_nodes()<100 :
    G = graphs[key]
    print("Graph: {}".format(key))
    print("------------------------")
    check_monotonicity(G,number_node)
    # print("Number of Nodes: {}".format(G.number_of_nodes()))
    # print("Number of Edges: {}".format(G.number_of_edges()))
    print("------------------------")


Graph: ca-sandi_auths
------------------------
0st round : compute score for node 20 , remove (48, 30) edge , meet condition? True
1st round : compute score for node 81 , remove (69, 53) edge , meet condition? True
2st round : compute score for node 83 , remove (75, 3) edge , meet condition? True
3st round : compute score for node 47 , remove (59, 45) edge , meet condition? True
4st round : compute score for node 33 , remove (43, 36) edge , meet condition? True
5st round : compute score for node 79 , remove (44, 33) edge , meet condition? True
6st round : compute score for node 32 , remove (53, 43) edge , meet condition? True
7st round : compute score for node 35 , remove (77, 70) edge , meet condition? True
8st round : compute score for node 30 , remove (48, 30) edge , meet condition? True
9st round : compute score for node 65 , remove (65, 14) edge , meet condition? True
Success rate =  100%
------------------------
Graph: reptilia-tortoise-network-lm
------------------------
0st rou

axiom 2 : rank monotonicity

In [9]:
def check_rank_monotonicity(G, number_node):

    # test if C(v,G-e)<= C(u,G-e) then C(v,G) <= C(u,G) for 10 random node
    check_array = [0, 0]
    for i in range(number_node):
        random_node = sample(list(G.nodes()),2)
        removed_edge = choice(list(G.edges()))
        score_v = count_tree(G, random_node[0])
        score_u = count_tree(G, random_node[1])
        G_temp = G.copy()
        G_temp.remove_edge(*removed_edge)
        score_v_without = count_tree(G_temp, random_node[0])
        score_u_without = count_tree(G_temp, random_node[1])
        check = (score_v_without <= score_u_without and score_v <= score_u) or (score_u_without <= score_v_without and score_u <= score_v)
        if check:
            check_array[0] += 1
        else:
            check_array[1] += 1
        print(str(i) + "st round : compute score for nodes " + str(random_node) + " , remove " + str(
            removed_edge) + " edge , meet condition? " + str(check))
    print("Success rate =  " + str(round((check_array[0] / (check_array[0] + check_array[1])) * 100)) + "%")


test axiom 2

In [10]:
number_node = 10
for key, value in graphs.items():
  if graphs[key].number_of_nodes()<100 :
    G = graphs[key]
    print("Graph: {}".format(key))
    print("------------------------")
    check_rank_monotonicity(G,number_node)
    # print("Number of Nodes: {}".format(G.number_of_nodes()))
    # print("Number of Edges: {}".format(G.number_of_edges()))
    print("------------------------")


Graph: ca-sandi_auths
------------------------
0st round : compute score for node [5, 59] , remove (73, 36) edge , meet condition? True
1st round : compute score for node [32, 42] , remove (33, 12) edge , meet condition? True
2st round : compute score for node [17, 10] , remove (74, 38) edge , meet condition? True
3st round : compute score for node [42, 3] , remove (66, 32) edge , meet condition? True
4st round : compute score for node [63, 41] , remove (73, 36) edge , meet condition? True
5st round : compute score for node [55, 66] , remove (41, 27) edge , meet condition? True
6st round : compute score for node [74, 62] , remove (69, 43) edge , meet condition? True
7st round : compute score for node [61, 43] , remove (60, 49) edge , meet condition? True
8st round : compute score for node [70, 79] , remove (26, 2) edge , meet condition? True
9st round : compute score for node [16, 15] , remove (75, 3) edge , meet condition? True
Success rate =  100%
------------------------
Graph: rept